# 运算符重载

Python支持运算符重载。并且通过对运算符重载施加一些限制，Python的运算符重载特性在灵活性、可用性以及安全性之间取得了一定的平衡。

具体来说，Python对运算符重载有如下限制：
1. 不能重载内置类型的运算符
2. 不能新建运算符
3. 某些运算符不能重载 —— is、and、or以及not（位运算符&、|以及~可以重载）

从实际使用中来看，上述规则实际上和Python特别强调duck type是相互协调的 —— Python中运算符重载功能的实现基于自定义类中对应方法的实现，即若想重载某一个运算符，则需要实现对应的方法。

由于没有办法覆写内置类型某些方法的实现，因此无法重载内置类型的运算符。

不能重载的运算符则不是通过上述机制运作的，因此也没有办法重载。下述例子查看了进行and、or以及not运算的字节码。显然，这些运算符并没有调用任何方法。and被翻译为字节码JUMP_IF_FALSE_OR_POP，or被翻译为JUMP_IF_TRUE_OR_POP，not则被翻译为UNARY_NOT。

In [29]:
# 尝试覆盖list类型的__len__方法
try:
    list.__len__ = 42
except TypeError:
    print("TypeError")

# 查看and语句的字节码
import dis
dis.opmap
def and_op_test(value_1, value_2):
    return not ((value_1<0 and value_2<0) or (value_1 != value_2))

dis.dis(and_op_test)

TypeError
 11           0 LOAD_FAST                0 (value_1)
              2 LOAD_CONST               1 (0)
              4 COMPARE_OP               0 (<)
              6 POP_JUMP_IF_FALSE       16
              8 LOAD_FAST                1 (value_2)
             10 LOAD_CONST               1 (0)
             12 COMPARE_OP               0 (<)
             14 JUMP_IF_TRUE_OR_POP     22
        >>   16 LOAD_FAST                0 (value_1)
             18 LOAD_FAST                1 (value_2)
             20 COMPARE_OP               3 (!=)
        >>   22 UNARY_NOT
             24 RETURN_VALUE


## 一元运算符的重载

由于一元运算符的操作数仅有一个，因此一元运算符的重载非常简单。和其他内置方法的重载一样，一元运算符的重载仅需要实现相应的特殊方法即可。

* -：一元取负算数运算符  .\_\_neg\_\_(self)
* +：一元取正算数运算符  .\_\_pos\_\_(self)
* ~：一元按位取反运算符  .\_\_invert\_\_(self)

值得注意的是，在进行上述操作后通常希望返回的结果和操作数有相同的类型。当然，这一条也不是绝对的，有些时候操作的结果和操作数的类型不同。但是无论如何，至少要保证运算返回的结果能够继续进行后续的运算（对于一个向量类Vector，若进行取反操作后返回的是一个列表，后续仅有Vector才能进行的操作就无法执行了，例如取模运算、转换为极坐标等）。

此外，一元运算符应当返回一个新对象。

## 二元运算符的重载

有相当一部分运算符是二元运算符。二元运算符涉及到两个操作数，因此相较于一元运算符，其重载也更复杂。具体来说，二元运算符的重载需要解决下述两种情况：
* self [op] others
* others [op] self

二元运算符需要两个操作数，并且没有办法保证这两个操作数的类型相同，因此需要设计有效的分派机制来尽可能求得运算结果。本章有一个非常好的流程图来描述这一分派过程，详情见本书第13章第3小节，下面简要叙述这一分派过程（以+为例）：

1. 取得运算符和两个操作数
2. 尝试调用左操作数的\_\_add\_\_
3. 若\_\_add\_\_调用成功，计算结果，否则返回NotImplemented并且跳转第5步
4. 若\_\_add\_\_返回NotImplemented，跳转第5步，否则跳转8步
5. 尝试调用右操作数的\_\_radd\_\_
6. 若\_\_radd\_\_调用成功，计算结果，否则raise TypeError
7. 若\_\_radd\_\_返回NotImplemented，raise TypeError，否则跳转8步
8. 返回计算结果

上述分派机制是有意义的，能够最大程度上保证求得运算结果。对内置类型和自定义类型的运算，上述分派过程尤其重要。

### 设计有效的报错信息

重载得到的二元运算符不可能涵盖所有类型，因此当涉及到不支持的类型时应当返回有利于debug的错误信息：“unsupported operand type(s) for {op}: {typeⅠ} and {typeⅡ}”，即当前运算不支持typeⅠ和typeⅡ之间的运算。

因此若运算错误，应当捕捉运算过程中出现的所有错误并返回上述的标准错误。下述是这一处理的“伪”代码：
```Python
def __add__(self, other):
    try:
        # 执行__add__的具体操作
        ...
    except TypeError:
        return NotImplemented

def __radd__(self, other):
    return self, other
```

通过try/except捕捉错误并在报错时返回NotImplemented。若解释器尚未尝试调用\_\_radd\_\_，则会尝试调用右操作数的\_\_radd\_\_，若依然返回NotImplemented，Python会raise TypeError并且返回标准错误信息。

## 比较运算符的特殊性

比较运算符也是一类二元运算符，但是有其特殊性：

1. 对于相等性比较（==以及!=），正反向均是调用同一个函数，只不过参数对调（a.\_\_eq\_\_(b)  b.\_\_eq\_\_(a)）
2. 对于排序（>, <, >=, <=），没有创建对应的反向特殊方法，而是调用反向逻辑对应的特殊方法（例如对于运算符>，其反向逻辑是运算符<）

|运算符|正向调用|反向调用|后备机制|
|:-:|:-:|:-:|:-:|
|a == b|a.\_\_eq\_\_(b)|b.\_\_eq\_\_(a)|return id(a) == id(b)|
|a != b|a.\_\_ne\_\_(b)|b.\_\_ne\_\_(a)|return not(a == b)|
|a >  b|a.\_\_gt\_\_(b)|b.\_\_lt\_\_(a)|raise TypeError|
|a <  b|a.\_\_lt\_\_(b)|b.\_\_gt\_\_(a)|raise TypeError|
|a >= b|a.\_\_ge\_\_(b)|b.\_\_le\_\_(a)|raise TypeError|
|a <= b|a.\_\_ge\_\_(b)|b.\_\_le\_\_(a)|raise TypeError|

## 增量赋值运算符的特殊性

不同于上述的运算符返回一个新对象，增量赋值运算符会进行原位操作。通常只要实现了对应的二元运算的特殊方法（\_\_add\_\_、\_\_sub\_\_等），增量赋值运算符就能够正常工作，此时增量赋值运算符仅是一种语法糖，其结果等同于普通的二元运算符。若想要实现真正的增量赋值运算符的重载，需要实现例如\_\_iadd\_\_、\_\_isub\_\_等方法。

真正的增量赋值运算符要求进行原位操作，因此结果的类型是可预见的，即结果的类型和第一个操作数的类型相同。从这个角度来说，增量赋值运算符对第二个操作数的类型更宽容 —— 第二个操作数的类型就算和第一个操作数的类型不同，仍可以尝试进行运算；对于一般的二元运算符，当两个操作数的类型不同时，除了极少数情况，建议直接raise TypeError，否则无论结果是哪种类型都可能引起困扰。

## 总结

1. Python支持运算符重载。并且通过对运算符重载施加一些限制，Python的运算符重载特性在灵活性、可用性以及安全性之间取得了一定的平衡。
2. 相较于二元运算符，由于一元运算符仅涉及到一个操作数，因此一元运算符的重载更简单，仅需要实现对应的特殊方法即可
3. 对于二元运算符，为了尽可能求得运算结果，Python有一整套的分派机制来执行二元运算。具体来说，Python会首先尝试调用正向的特殊方法，例如\_\_add\_\_、\_\_sub\_\_；若返回了特殊的NotImplemented，则调用反向的特殊方法，例如\_\_radd\_\_、\_\_rsub\_\_。
4. 若二元运算的两个操作数类型不同并且无法计算得到结果，为了debug方便，通常需要捕获错误并且返回便于debug的错误消息。对此，本章提出了两种方法，其一是使用try/except直接尝试运行然后捕获错误，在捕获错误后返回NotImplemented以便于返回标准错误；其二是使用isinstance直接测试第二个操作数是否符合要求，若不符合要求则返回NotImplemented以便于返回标准错误。值得注意的是，isinstance应当测试抽象基类，而不应该测试具体类，这样操作兼具灵活性和安全性。
5. 比较运算符虽然是二元运算符，但是这些运算符有其特殊性。具体来说，相等性比较从不抛出TypeError，最终会由id()兜底；而其他比较运算符对应的反向特殊方法逻辑不同于一般的二元运算符。
6. 上述运算均会返回一个新对象，增量赋值运算符的运算结果则是进行原位修改。相较于一般的二元运算符，增量赋值运算符由于可以预知运算结果的类型，因此这些操作符对操作数的要求更宽松。相比之下，一般的二元运算符，除特殊情况外，通常要求两个操作数的类型相同。